Consider a real-world diagonalisation algorithm:

In [1]:
using LinearAlgebra

qrortho(X::Array)   = Array(qr(X).Q)
qrortho(X, Y)       = qrortho(X - Y * Y'X)

function rayleigh_ritz(X::Array, AX::Array, N)
    F = eigen(Hermitian(X'AX))
    F.values[1:N], F.vectors[:,1:N]
end

function davidson(A, SS::AbstractArray; tol=1e-5, maxsubspace=8size(SS, 2))
    m = size(SS, 2)
    for i in 1:100
        Ass = A * SS
        rvals, rvecs = rayleigh_ritz(SS, Ass, m)
        Ax = Ass * rvecs

        R = Ax - SS * rvecs * Diagonal(rvals)
        if norm(R) < tol
            return rvals, SS * rvecs
        end

        println(i, "  ", size(SS, 2), "  ", norm(R))

        # Use QR to orthogonalise the subspace.
        if size(SS, 2) + m > maxsubspace
            SS = qrortho([SS*rvecs R])
        else
            SS = qrortho([SS       R])
        end
    end
    error("not converged.")
end

davidson (generic function with 1 method)

In [2]:
nev = 2
A = randn(50, 50); A = A + A' + 5I;

# Generate two random orthogonal guess vectors
x0 = qrortho(randn(size(A, 2), nev))

# Run the problem
davidson(A, x0)

1  2  12.131738040889848
2  4  9.033730413134899
3  6  6.370318151651235
4  8  5.030137853688143
5  10  4.351018947944646
6  12  2.9266678130865933
7  14  1.9337339482848037
8  16  1.2495631648926946
9  4  0.8447225113063361
10  6  0.6618682514403892
11  8  0.42872402272861015
12  10  0.2787310748024603
13  12  0.15045593185087466
14  14  0.0816740777320898
15  16  0.052235442856243965
16  4  0.03307463920326185
17  6  0.03332854620655638
18  8  0.01815933730373792
19  10  0.010568721221358626
20  12  0.006566789553707762
21  14  0.003907532137333239
22  16  0.003047648504664202
23  4  0.0019946035413008116
24  6  0.0015106601301103815
25  8  0.0009819152883112925
26  10  0.0006652219390769453
27  12  0.000389131358895756
28  14  0.00018291786432362802
29  16  0.00016847588984320776
30  4  0.00011224620732879244
31  6  0.00011393277392875974
32  8  6.833875871827651e-5
33  10  4.0123844871362163e-5
34  12  2.4016409444087852e-5
35  14  1.4376666012754893e-5
36  16  1.2434828237220205e-

([-12.419375113217214, -11.965348928710366], [0.01261951720687232 -0.3496366053621342; 0.13294188632687517 0.0005144764024024648; … ; 0.29476659156116997 -0.02278700010588038; 0.0006078125306643152 -0.08092581000529563])

In [3]:
# Mixed precision!
using GenericLinearAlgebra

λ, v = davidson(Matrix{Float32}(A), Float32.(x0), tol=1e-3)
println()
λ, v = davidson(Matrix{Float64}(A), v, tol=1e-13)
println()
λ, v = davidson(Matrix{BigFloat}(A), v, tol=1e-25)
λ

1  2  12.131738
2  4  9.033732
3  6  6.370319
4  8  5.0301423
5  10  4.3510165
6  12  2.926664
7  14  1.9337329
8  16  1.2495672
9  4  0.84472376
10  6  0.66186553
11  8  0.4287225
12  10  0.27873093
13  12  0.15045314
14  14  0.0816719
15  16  0.052233797
16  4  0.033073798
17  6  0.03332756
18  8  0.018159874
19  10  0.010568154
20  12  0.006565463
21  14  0.0039067254
22  16  0.0030478162
23  4  0.0019938245
24  6  0.0015088111

1  2  0.0009815610373936645
2  4  0.0006557910964309086
3  6  0.000460922574973398
4  8  0.00033203029081309694
5  10  0.0002802797726259403
6  12  0.00014189413688395978
7  14  5.798272544612427e-5
8  16  5.02974399356079e-5
9  4  2.9565957978160854e-5
10  6  2.606388265944082e-5
11  8  1.608273399817224e-5
12  10  8.16005880193561e-6
13  12  5.274604962968202e-6
14  14  3.528944050780912e-6
15  16  2.995051660581015e-6
16  4  2.03118050979857e-6
17  6  1.5386391398358925e-6
18  8  1.1990759180094675e-6
19  10  8.122430821399884e-7
20  12  4.701186098039528

2-element Vector{BigFloat}:
 -12.41937511321919516318481274797025859363679706589028056819868884905926849380504
 -11.96534892872113649906936247524945443412555661878925210978919616724282201895872

In [4]:
using SparseArrays
nev = 2
spA = sprandn(100, 100, 0.3); spA = spA + spA' + 2I
spx0 = randn(size(spA, 2), nev)
spx0 = Array(qr(spx0).Q)

davidson(spA, spx0, tol=1e-6)

1  2  10.637541023248037
2  4  7.198521961678987
3  6  5.376976563155802
4  8  5.003687322773134
5  10  3.5764423558481258
6  12  2.3557583787540683
7  14  1.5267513007217737
8  16  1.2258398054151183
9  4  0.8017632017609047
10  6  0.6517093300550234
11  8  0.5811845765469917
12  10  0.5479032637066457
13  12  0.6988361954267349
14  14  1.0555578985834444
15  16  0.9732559921767933
16  4  0.629288361918641
17  6  0.4899207056864191
18  8  0.37442510425964187
19  10  0.2778202094875601
20  12  0.18205168807988903
21  14  0.11492652841734978
22  16  0.0844988102355338
23  4  0.0837898224547055
24  6  0.08934058984175938
25  8  0.07767368897453182
26  10  0.06551812047162994
27  12  0.05945276707504045
28  14  0.04628183517346407
29  16  0.040709046704606
30  4  0.023762528282998453
31  6  0.016980593272519284
32  8  0.012633492315706483
33  10  0.009620512225319242
34  12  0.006931478865046648
35  14  0.00487878889954326
36  16  0.004056221107432724
37  4  0.003847482486979358
38  6  0.

([-13.460486634942196, -12.380663428278165], [-0.07627869551786422 0.014701301308350163; -0.12787230863110488 0.09587885948437846; … ; 0.07552139814055045 0.06926072936623444; -0.12732443094300236 -0.014261042666499333])

In [5]:
# ... runs with GPUs !
using CUDA

qrortho(X::CuArray) = CuArray(qr(X).Q)

function rayleigh_ritz(X::CuArray, AX::CuArray, N)
    values, vectors = CUDA.CUSOLVER.syevd!('V', 'U', X'AX)
    values[1:N], vectors[:,1:N]
end

┌ Info: Precompiling CUDA [052768ef-5323-5732-b1bb-66c8b64840ba]
└ @ Base loading.jl:1317


rayleigh_ritz (generic function with 2 methods)

In [6]:
davidson(cu(A), cu(x0))

1  2  12.131738
2  4  9.0337305
3  6  6.3703203
4  8  5.030137
5  10  4.3510175
6  12  2.9266672
7  14  1.9337337
8  16  1.249565
9  4  0.844723
10  6  0.6618678
11  8  0.4287245
12  10  0.2787323
13  12  0.15045786
14  14  0.08167462
15  16  0.05223599
16  4  0.03307448
17  6  0.033328887
18  8  0.018160751
19  10  0.010571146
20  12  0.006567974
21  14  0.0039073657
22  16  0.0030456078
23  4  0.0019940762
24  6  0.0015096914
25  8  0.0009825465
26  10  0.0006650116
27  12  0.00038867083
28  14  0.00018202147
29  16  0.00017028602
30  4  0.00010910879
31  6  0.000105263694
32  8  6.512129e-5
33  10  4.1167437e-5
34  12  2.371476e-5
35  14  1.3068939e-5
36  16  1.49934785e-5


(Float32[-12.419375, -11.965352], Float32[0.012619994 0.3496365; 0.13294193 -0.00051462353; … ; 0.29476655 0.02278679; 0.0006078709 0.08092586])